# **Assembly Workshop**
---
# Priority Resource

In [253]:
import simpy
import random
import numpy as np

In [254]:
jobs = {}

stats = {'cycle_times':
         {
             1:[],
             2:[]
         }
         }

In [255]:
def job_arrival(env, workers):
    i=0
    while True:
        i+=1
        yield env.timeout(random.expovariate(1/10))
        job_prio = np.random.choice([1,2], p=[0.2, 0.8])
        name = i
        jobs[name] = env.now

        env.process(process_job(env, name, job_prio, workers))

        if i>=100:
            break

def process_job(env, name, job_prio, workers):
    with workers.request(priority=job_prio) as req:
        yield req
        yield env.timeout(20) # 20 minutes

    print(f'Job {name} with priority {job_prio} finished at {env.now}')
    jobs[name] = env.now - jobs[name]
    stats['cycle_times'][job_prio].append(jobs[name])

    print(f'Avg CT of Job Type 1: {np.mean(stats["cycle_times"][1]): .2f}')
    print(f'Avg CT of Job Type 2: {np.mean(stats["cycle_times"][2]): .2f}')

In [256]:
env = simpy.Environment()

workers = simpy.PriorityResource(env, capacity=1)

env.process(job_arrival(env, workers))

env.run()

Job 1 with priority 2 finished at 28.03443193092484
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  20.00
Job 2 with priority 2 finished at 48.03443193092484
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  25.26
Job 3 with priority 2 finished at 68.03443193092484
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  29.83
Job 4 with priority 2 finished at 88.03443193092484
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  36.82
Job 5 with priority 2 finished at 108.03443193092484
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  44.36
Job 6 with priority 2 finished at 128.03443193092482
Avg CT of Job Type 1:  nan
Avg CT of Job Type 2:  50.80
Job 11 with priority 1 finished at 148.03443193092482
Avg CT of Job Type 1:  28.89
Avg CT of Job Type 2:  50.80
Job 7 with priority 2 finished at 168.03443193092482
Avg CT of Job Type 1:  28.89
Avg CT of Job Type 2:  59.55
Job 8 with priority 2 finished at 188.03443193092482
Avg CT of Job Type 1:  28.89
Avg CT of Job Type 2:  66.19
Job 19 with prio

/opt/miniconda3/envs/Sim/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/Sim/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


---
# Store

In [257]:
# list of random vegetables
veggies = ['carrot', 'potato', 'onion', 'cabbage']

# Producer process
def producer(env, store):
    while True:
        yield env.timeout(2)
        vegetable = random.choice(veggies)
        yield store.put(vegetable)
        print(f"Produced {vegetable} at time {env.now : .2f}.")

def customer(name, env, store):
    print(f"Customer {name} arrives and request vegetable at time {env.now : .2f}.")
    vegetable = yield store.get()
    print(f"Customer {name} got a {vegetable} at {env.now : .2f} and leaves.")

def customer_generator(env, store):
    i = 0
    while True:
        yield env.timeout(1)
        env.process(customer(name=i, env=env, store=store))
        i += 1

In [258]:
env = simpy.Environment()

store = simpy.Store(env, capacity=  5)

# Start producer
env.process(producer(env, store))

# Start customer generator process
env.process(customer_generator(env, store))

env.run(until=100)

Customer 0 arrives and request vegetable at time  1.00.
Customer 1 arrives and request vegetable at time  2.00.
Produced carrot at time  2.00.
Customer 0 got a carrot at  2.00 and leaves.
Customer 2 arrives and request vegetable at time  3.00.
Customer 3 arrives and request vegetable at time  4.00.
Produced potato at time  4.00.
Customer 1 got a potato at  4.00 and leaves.
Customer 4 arrives and request vegetable at time  5.00.
Customer 5 arrives and request vegetable at time  6.00.
Produced cabbage at time  6.00.
Customer 2 got a cabbage at  6.00 and leaves.
Customer 6 arrives and request vegetable at time  7.00.
Customer 7 arrives and request vegetable at time  8.00.
Produced cabbage at time  8.00.
Customer 3 got a cabbage at  8.00 and leaves.
Customer 8 arrives and request vegetable at time  9.00.
Customer 9 arrives and request vegetable at time  10.00.
Produced cabbage at time  10.00.
Customer 4 got a cabbage at  10.00 and leaves.
Customer 10 arrives and request vegetable at time  

---
# Priority Store

In [259]:
# list of random vegetables
veggies = ['carrot', 'potato', 'onion', 'cabbage']

# Producer process
def producer(env, store):
    while True:
        yield env.timeout(2)
        vegetable = random.choice(veggies)
        yield store.put(vegetable)
        print(f"Produced {vegetable} at time {env.now : .2f}.")

def customer(name, env, store):
    req_veg = random.choice(veggies)
    print(f"Customer {name} arrives and requests vegetable {req_veg} at time {env.now : .2f}.")
    vegetable = yield store.get(lambda vegetable: vegetable == req_veg)
    print(f"Customer {name} got a {vegetable} at {env.now : .2f} and leaves.")

def customer_generator(env, store):
    i = 0
    while True:
        yield env.timeout(1)
        env.process(customer(name=i, env=env, store=store))
        i += 1

In [260]:
env = simpy.Environment()

store = simpy.FilterStore(env, capacity=  5)

# Start producer
env.process(producer(env, store))

# Start customer generator process
env.process(customer_generator(env, store))

env.run(until=100)

Customer 0 arrives and requests vegetable carrot at time  1.00.
Customer 1 arrives and requests vegetable carrot at time  2.00.
Produced cabbage at time  2.00.
Customer 2 arrives and requests vegetable potato at time  3.00.
Customer 3 arrives and requests vegetable onion at time  4.00.
Produced onion at time  4.00.
Customer 3 got a onion at  4.00 and leaves.
Customer 4 arrives and requests vegetable potato at time  5.00.
Customer 5 arrives and requests vegetable cabbage at time  6.00.
Produced potato at time  6.00.
Customer 2 got a potato at  6.00 and leaves.
Customer 5 got a cabbage at  6.00 and leaves.
Customer 6 arrives and requests vegetable cabbage at time  7.00.
Customer 7 arrives and requests vegetable onion at time  8.00.
Produced potato at time  8.00.
Customer 4 got a potato at  8.00 and leaves.
Customer 8 arrives and requests vegetable onion at time  9.00.
Customer 9 arrives and requests vegetable carrot at time  10.00.
Produced carrot at time  10.00.
Customer 0 got a carrot 

---
# Filter Store

In [261]:
# Producer process
def producer(env, store):
    while True:
        yield env.timeout(2)
        vegetable_name = 'potato'
        expires_in = env.now + random.randint(0, 10)
        vegetable = simpy.PriorityItem(priority=expires_in, item=vegetable_name)
        yield store.put(vegetable)
        print(f"Produced {vegetable} at time {env.now : .2f}.")

def customer(name, env, store):
    print(f"Customer {name} arrives and request potato at time {env.now : .2f}.")
    vegetable = yield store.get()
    print(f"Customer {name} got a {vegetable} at {env.now : .2f} and leaves.")

def customer_generator(env, store):
    i = 0
    while True:
        yield env.timeout(1)
        env.process(customer(name=i, env=env, store=store))
        i += 1

In [262]:
env = simpy.Environment()

store = simpy.PriorityStore(env, capacity=  5)

# Start producer
env.process(producer(env, store))

# Start customer generator process
env.process(customer_generator(env, store))

env.run(until=100)

Customer 0 arrives and request potato at time  1.00.
Customer 1 arrives and request potato at time  2.00.
Produced PriorityItem(priority=4, item='potato') at time  2.00.
Customer 0 got a PriorityItem(priority=4, item='potato') at  2.00 and leaves.
Customer 2 arrives and request potato at time  3.00.
Customer 3 arrives and request potato at time  4.00.
Produced PriorityItem(priority=7, item='potato') at time  4.00.
Customer 1 got a PriorityItem(priority=7, item='potato') at  4.00 and leaves.
Customer 4 arrives and request potato at time  5.00.
Customer 5 arrives and request potato at time  6.00.
Produced PriorityItem(priority=8, item='potato') at time  6.00.
Customer 2 got a PriorityItem(priority=8, item='potato') at  6.00 and leaves.
Customer 6 arrives and request potato at time  7.00.
Customer 7 arrives and request potato at time  8.00.
Produced PriorityItem(priority=16, item='potato') at time  8.00.
Customer 3 got a PriorityItem(priority=16, item='potato') at  8.00 and leaves.
Custom

---
# Custom Event

In [263]:
def coffee_machine(env):
    while True:
        print(f"Coffee machine ready to use at {env.now : .2f}.")
        random_running_out_of_coffee = random.uniform(60, 180) # minutes
        yield env.timeout(random_running_out_of_coffee)
        refill = env.event()
        print(f"Coffee over! Call operator at time {env.now : .2f}.")
        env.process(refilling(env, refill))
        yield refill

def refilling(env, refill):
    refill_time = 15 #minutes
    yield env.timeout(refill_time)
    refill.succeed()
    clean_floor = 5 #minutes
    yield env.timeout(clean_floor)
    print(f"Floor cleaned. Operator leaves at {env.now : .2f}.")

In [264]:
env = simpy.Environment()

env.process(coffee_machine(env))

env.run(until=24*60)

Coffee machine ready to use at  0.00.
Coffee over! Call operator at time  77.94.
Coffee machine ready to use at  92.94.
Floor cleaned. Operator leaves at  97.94.
Coffee over! Call operator at time  241.67.
Coffee machine ready to use at  256.67.
Floor cleaned. Operator leaves at  261.67.
Coffee over! Call operator at time  342.99.
Coffee machine ready to use at  357.99.
Floor cleaned. Operator leaves at  362.99.
Coffee over! Call operator at time  491.99.
Coffee machine ready to use at  506.99.
Floor cleaned. Operator leaves at  511.99.
Coffee over! Call operator at time  634.48.
Coffee machine ready to use at  649.48.
Floor cleaned. Operator leaves at  654.48.
Coffee over! Call operator at time  714.67.
Coffee machine ready to use at  729.67.
Floor cleaned. Operator leaves at  734.67.
Coffee over! Call operator at time  824.78.
Coffee machine ready to use at  839.78.
Floor cleaned. Operator leaves at  844.78.
Coffee over! Call operator at time  936.43.
Coffee machine ready to use at  

---
# Preemptive Resource

## Approach One (Without Preemptive Resource)

In [265]:
def coffee_machine(env):
    while True:
        print(f'Coffee machine ready to use at {env.now : .2f}.')
        random_interarrival_time = random.uniform(60, 180)
        yield env.timeout(random_interarrival_time)
        print(f'Coffee over! Call worker at {env.now : .2f}.')
        cleaning.interrupt()
        global refilled_event
        yield refilled_event
        refilled_event = env.event()

def clean_floor(env, worker):
    time_to_clean = 120
    while True:
        try:
            while time_to_clean:
                print(f'Cleaning started {env.now : .2f} amd time to clean = {time_to_clean : .2f}.')
                with worker.request() as req:
                    yield req
                    start = env.now
                    yield env.timeout(time_to_clean)
                    time_to_clean = 120
        except simpy.Interrupt:
            current_time = env.now
            time_to_clean -= current_time - start
            if time_to_clean == 0:
                time_to_clean = 120
            print(f'Cleaning process interrupted at {env.now : .2f} and time left to clean = {time_to_clean : .2f}.')
            yield env.process(refilling(env, worker))

def refilling(env, worker):
    with worker.request() as req:
        yield req
        yield env.timeout(15)
        print(f'Refilled.Machine ready to use at {env.now : .2f}.')
        global refilled_event
        refilled_event.succeed()

env = simpy.Environment()
refilled_event = env.event()
worker = simpy.Resource(env, capacity=1)
env.process(coffee_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until=24*60)

Coffee machine ready to use at  0.00.
Cleaning started  0.00 amd time to clean =  120.00.
Coffee over! Call worker at  119.28.
Cleaning process interrupted at  119.28 and time left to clean =  0.72.
Refilled.Machine ready to use at  134.28.
Coffee machine ready to use at  134.28.
Cleaning started  134.28 amd time to clean =  0.72.
Cleaning started  135.00 amd time to clean =  120.00.
Cleaning started  255.00 amd time to clean =  120.00.
Coffee over! Call worker at  257.76.
Cleaning process interrupted at  257.76 and time left to clean =  117.24.
Refilled.Machine ready to use at  272.76.
Coffee machine ready to use at  272.76.
Cleaning started  272.76 amd time to clean =  117.24.
Coffee over! Call worker at  366.58.
Cleaning process interrupted at  366.58 and time left to clean =  23.42.
Refilled.Machine ready to use at  381.58.
Coffee machine ready to use at  381.58.
Cleaning started  381.58 amd time to clean =  23.42.
Cleaning started  405.00 amd time to clean =  120.00.
Coffee over! 

## Approach 2 (Preemptive Resource)

In [266]:
def coffee_machine(env):
    while True:
        print(f'Coffee machine ready to use at {env.now : .2f}.')
        random_interarrival_time = random.uniform(60, 180)
        yield env.timeout(random_interarrival_time)
        print(f'Coffee over! Call worker at {env.now : .2f}.')
        yield env.process(refilling(env, worker))

def clean_floor(env, worker):
    time_to_clean = 120
    while True:
        try:
            while time_to_clean:
                print(f'Cleaning started {env.now : .2f} amd time to clean = {time_to_clean : .2f}.')
                with worker.request(priority = 1) as req:
                    yield req
                    start = env.now
                    yield env.timeout(time_to_clean)
                    print(f'Cleaning finished {env.now : .2f} amd time to clean = {time_to_clean : .2f}.')
                    time_to_clean = 120
        except simpy.Interrupt:
            current_time = env.now
            time_to_clean -= current_time - start
            if time_to_clean == 0:
                time_to_clean = 120
            print(f'Cleaning process interrupted at {env.now : .2f} and time left to clean = {time_to_clean : .2f}.')

def refilling(env, worker):
    with worker.request(priority = 0) as req:
        yield req
        yield env.timeout(15)
        print(f'Refilled.Machine ready to use at {env.now : .2f}.')

env = simpy.Environment()
worker = simpy.PreemptiveResource(env, capacity=1)
env.process(coffee_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until=24*60)

Coffee machine ready to use at  0.00.
Cleaning started  0.00 amd time to clean =  120.00.
Coffee over! Call worker at  69.76.
Cleaning process interrupted at  69.76 and time left to clean =  50.24.
Cleaning started  69.76 amd time to clean =  50.24.
Refilled.Machine ready to use at  84.76.
Coffee machine ready to use at  84.76.
Cleaning finished  135.00 amd time to clean =  50.24.
Cleaning started  135.00 amd time to clean =  120.00.
Cleaning finished  255.00 amd time to clean =  120.00.
Cleaning started  255.00 amd time to clean =  120.00.
Coffee over! Call worker at  263.08.
Cleaning process interrupted at  263.08 and time left to clean =  111.92.
Cleaning started  263.08 amd time to clean =  111.92.
Refilled.Machine ready to use at  278.08.
Coffee machine ready to use at  278.08.
Cleaning finished  390.00 amd time to clean =  111.92.
Cleaning started  390.00 amd time to clean =  120.00.
Coffee over! Call worker at  402.15.
Cleaning process interrupted at  402.15 and time left to cle